<a href="https://colab.research.google.com/github/noswad/Python/blob/master/ANT_ultimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ANT加強版

1.安裝必要的庫

In [ ]:
!pip install yfinance sqlalchemy

2. 下載TWSE和OTC股票清單，並存儲到CSV檔案

In [ ]:
import requests
import pandas as pd

# 下載TWSE上市股票清單
twse_url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=2'
twse_response = requests.get(twse_url)
twse_df = pd.read_html(twse_response.text)[0]
twse_df.columns = twse_df.iloc[0]
twse_df = twse_df[1:]
twse_df = twse_df[['有價證券代號及名稱', '國際證券辨識號碼(ISIN Code)', '上市日', '市場別', '產業別', 'CFICode', '備註']]
twse_df.columns = ['stock_id_name', 'ISIN', '上市日', '市場別', '產業別', 'CFICode', '備註']
twse_df['stock_id'] = twse_df['stock_id_name'].apply(lambda x: x.split('　')[0] if '　' in x else x)
twse_df['name'] = twse_df['stock_id_name'].apply(lambda x: x.split('　')[1] if '　' in x else '')

# 下載OTC上櫃股票清單
otc_url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=4'
otc_response = requests.get(otc_url)
otc_df = pd.read_html(otc_response.text)[0]
otc_df.columns = otc_df.iloc[0]
otc_df = otc_df[1:]
otc_df = otc_df[['有價證券代號及名稱', '國際證券辨識號碼(ISIN Code)', '上市日', '市場別', '產業別', 'CFICode', '備註']]
otc_df.columns = ['stock_id_name', 'ISIN', '上市日', '市場別', '產業別', 'CFICode', '備註']
otc_df['stock_id'] = otc_df['stock_id_name'].apply(lambda x: x.split('　')[0] if '　' in x else x)
otc_df['name'] = otc_df['stock_id_name'].apply(lambda x: x.split('　')[1] if '　' in x else '')

# 合併清單並標記是否為上櫃類股
twse_df['is_otc'] = False
otc_df['is_otc'] = True
stock_list = pd.concat([twse_df, otc_df], ignore_index=True).drop_duplicates(subset=['stock_id'])

# 儲存為CSV檔案
stock_list.to_csv('/content/stock_list.csv', index=False, encoding='utf-8-sig')


3. 從yfinance獲取近90天的數據並寫入SQLite資料庫

In [ ]:
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine, text
import time
from tqdm import tqdm

# 讀取合併後的台股代碼清單
stock_list = pd.read_csv('/content/stock_list.csv')

# 確保 stock_id 是字符串格式
stock_list['stock_id'] = stock_list['stock_id'].astype(str)

# 連接到 SQLite 資料庫
engine = create_engine('sqlite:///stock_data.db')
conn = engine.connect()

# 初始化 log 檔案
log_file_path = '/content/fetch_log.txt'
with open(log_file_path, 'w') as log_file:
    log_file.write("抓取數據日誌:\n")

# 設定請求間隔時間
SLEEP_INTERVAL = 0.8  # 每次請求後的等待時間（秒）

# 下載歷史數據並寫入資料庫
total_stocks = len(stock_list)
with tqdm(total=total_stocks, desc="抓取股票數據進度") as pbar:
    for i in stock_list.index:
        stock_id = stock_list.loc[i, 'stock_id']
        stock_id_full = stock_id.zfill(4) + ('.TWO' if stock_list.loc[i, 'is_otc'] else '.TW')
        log_message = f"抓取 {stock_id_full} 的數據"

        data = yf.Ticker(stock_id_full)
        df = data.history(period="3mo")  # 改為獲取近 3 個月的數據

        # 檢查是否成功獲取數據
        if len(df) < 60:  # 近 3 個月應有大約 60 個交易日，這裡設置為 60 以確保足夠的數據
            log_message = f"{stock_id_full} 數據不足，跳過此股票"
            with open(log_file_path, 'a') as log_file:
                log_file.write(log_message + "\n")
            pbar.update(1)
            continue

        if not df.empty:
            # 只保留需要的列
            df = df[['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']]
            df['stock_id'] = stock_id
            df.reset_index(inplace=True)
            df.to_sql('historical_data', conn, if_exists='append', index=False)
            log_message = f"{stock_id_full} 的數據已成功寫入資料庫"
            with open(log_file_path, 'a') as log_file:
                log_file.write(log_message + "\n")

        pbar.update(1)
        time.sleep(SLEEP_INTERVAL)  # 控制請求速率

# 查看資料庫中的數據
print("資料抓取完成，顯示前 5 筆資料:")
result = conn.execute(text("SELECT * FROM historical_data LIMIT 5")).fetchall()
for row in result:
    print(row)


抓取股票數據進度: 100%|██████████| 1846/1846 [27:48<00:00,  1.11it/s]

資料抓取完成，顯示前 5 筆資料:
('2024-02-29 00:00:00.000000', 271.5, 293.5, 270.5, 293.5, 7073000, 0.0, 0.0, '6231')
('2024-03-01 00:00:00.000000', 299.5, 318.0, 277.5, 312.0, 10913000, 0.0, 0.0, '6231')
('2024-03-04 00:00:00.000000', 315.0, 323.5, 302.5, 303.0, 5109000, 0.0, 0.0, '6231')
('2024-03-05 00:00:00.000000', 307.5, 307.5, 295.0, 296.5, 2286000, 0.0, 0.0, '6231')
('2024-03-06 00:00:00.000000', 290.5, 299.5, 286.0, 291.5, 2380000, 0.0, 0.0, '6231')


4. 從資料庫中查詢數據並進行篩選

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import yfinance as yf

# 連接到 SQLite 資料庫
engine = create_engine('sqlite:///stock_data.db')
conn = engine.connect()

# 查詢歷史數據
query = "SELECT * FROM historical_data"
df = pd.read_sql_query(query, conn)

# 轉換日期列為 datetime 類型
df['Date'] = pd.to_datetime(df['Date'])

# 確保 stock_id 是字符串格式
df['stock_id'] = df['stock_id'].astype(str)

# 載入股票清單
stock_list = pd.read_csv('/content/stock_list.csv')
stock_list['stock_id'] = stock_list['stock_id'].astype(str)

# 按 stock_id 分組並計算指標
results = []
stocks = df['stock_id'].unique()
for stock in stocks:
    stock_data = df[df['stock_id'] == stock].sort_values(by='Date')
    stock_data['Change'] = stock_data['Close'].pct_change()
    stock_data['Volume_Change'] = stock_data['Volume'].pct_change()

    last_15_days = stock_data.tail(15)

    # 動能條件：過去 15 天至少有 12 天上漲
    momentum_condition = (last_15_days['Change'] > 0).sum() >= 12

    # 檢查是否有足夠的數據和成交量不為零
    prev_15_days = stock_data.iloc[-30:-15]
    if len(prev_15_days) < 15 or prev_15_days['Volume'].mean() == 0:
        continue

    # 成交量條件：過去 15 天內成交量至少增加了 25%
    volume_condition = (last_15_days['Volume'].mean() / prev_15_days['Volume'].mean()) >= 1.25

    # 價格條件：過去 15 天內價格至少上漲 20%
    price_condition = (last_15_days['Close'].iloc[-1] / last_15_days['Close'].iloc[0]) >= 1.2

    if momentum_condition:
        if price_condition and volume_condition:
            level = "Perfect - 符合動能、成交量和價格要求。"
        elif price_condition:
            level = "Level 2 - 符合動能和價格要求。"
        elif volume_condition:
            level = "Level 3 - 符合動能和成交量要求。"
        else:
            level = "Level 1 - 符合動能要求。"
    else:
        level = "None"

    if level != "None":
        stock_name = stock_list.loc[stock_list['stock_id'] == stock, 'name'].values[0]
        results.append([stock, stock_name, level])

# 將結果轉換為 DataFrame
results_df = pd.DataFrame(results, columns=['stock_id', 'name', 'level'])

# 從 yfinance 獲取200MA數據並進行判斷
final_results = []
for index, row in results_df.iterrows():
    stock_id_full = row['stock_id'].zfill(4) + ('.TWO' if stock_list.loc[stock_list['stock_id'] == row['stock_id'], 'is_otc'].values[0] else '.TW')
    data = yf.Ticker(stock_id_full)
    df = data.history(period="1y")  # 獲取近 1 年的數據以計算200MA
    if len(df) < 200:
        continue  # 確保有足夠的數據計算200MA

    df['200MA'] = df['Close'].rolling(window=200).mean()
    if df['Close'].iloc[-1] > df['200MA'].iloc[-1]:
        final_results.append([row['stock_id'], row['name'], row['level']])

# 將最終結果轉換為 DataFrame
final_results_df = pd.DataFrame(final_results, columns=['stock_id', 'name', 'level'])

# 將結果寫入 CSV 檔案，使用 UTF-8 編碼格式
final_results_df.to_csv('/content/list.csv', index=False, encoding='utf-8-sig')

print("結果已寫入 list.csv 檔案。")


結果已寫入 list.csv 檔案。
